In [6]:
from pymongo import MongoClient
import matplotlib.pyplot as plt
import os
from sshtunnel import SSHTunnelForwarder
import numpy as np
import pandas as pd
import networkx as nx

In [2]:
MONGO_HOST = "193.196.55.231"
MONGO_DB = "twitter_db_DE"

server = SSHTunnelForwarder(
    MONGO_HOST,
    ssh_username="ubuntu",
    ssh_pkey="/home/tobias/.ssh/ghost_id_rsa",
    remote_bind_address=('127.0.0.1', 27017)
)

server.start()

client = MongoClient('127.0.0.1', server.local_bind_port) # server.local_bind_port is assigned local port
db = client[MONGO_DB]


In [29]:
data = db.cc_follows.find({}, {"id": 1, "following" : 1, "followed_by": 1, "seed":1, "username" : 1})
df = pd.DataFrame(list(data))


In [30]:
df

,_id,id,username,seed,following,followed_by
0,61bc94b7b89585883e4d463d,818876014390603776,FLOTUS45,1466829037645582341,[],"[1183975422863773696, 757578754210267136, 1281..."
1,61bc94b7b89585883e4d463e,1212806053907185664,TheLeoTerrell,1466829037645582341,[],"[933167783931768832, 69891324, 46979679, 48347..."
2,61bc94b7b89585883e4d463f,296508855,WegeInsLeben,1466829037645582341,"[933167783931768832, 2460335661, 251657704, 14...","[933167783931768832, 2460335661, 1337349377300..."
3,61bc94b7b89585883e4d4640,2913614824,BVG_Kampagne,1466829037645582341,"[493146582, 3083343255, 571984571, 22951215]","[1180966443963310081, 1280110927283699712, 758..."
4,61bc94b7b89585883e4d4641,1431512692943171586,Melanie03970811,1466829037645582341,[1434129791859535872],[1434129791859535872]
...,...,...,...,...,...,...
1084427,61cefe6c07664d890bef458e,1322793684207808514,Stefan43741005,1466829037645582341,[],[296714530]
1084428,61cefe6c07664d890bef458d,1308844459833122817,Kinkylitzchen,1466829037645582341,[2406535248],[2406535248]
1084429,61cefe6c07664d890bef4590,23765046,Apisticus,1466829037645582341,[],[1095718526323691520]
1084430,61cefe6c07664d890bef4591,844196171266560000,FrohlichHorst,1466829037645582341,[296714530],[]


In [67]:
G = nx.DiGraph()

In [53]:
edge_dict = {}
for index, row in df.iterrows():
    if index % 25000 == 0:
        print(index)
    for elem in row["following"]:
        G.add_edge(row["id"], elem, seed=row["seed"], username=row["username"])
        G.nodes[row["id"]]['seed'] = row["seed"]
        G.nodes[row["id"]]['username'] = row["username"]
    for elem in row["followed_by"]:
        G.add_edge(elem, row["id"], seed=row["seed"], username=row["username"])
        G.nodes[row["id"]]['seed'] = row["seed"]
        G.nodes[row["id"]]['username'] = row["username"]

0
25000
50000
75000
100000
125000
150000
175000
200000
225000
250000
275000
300000
325000
350000
375000
400000
425000
450000
475000
500000
525000
550000
575000
600000
625000
650000
675000
700000
725000
750000
775000
800000
825000
850000
875000
900000
925000
950000
975000
1000000
1025000
1050000
1075000


In [68]:
edge_dict = {}
for index, row in df.iterrows():
    if index % 25000 == 0:
        print(index)
    for elem in row["following"]:
        G.add_edge(row["id"], elem, seed=row["seed"], username=row["username"])
        G.nodes[row["id"]]['seed'] = row["seed"]
        G.nodes[row["id"]]['username'] = row["username"]

0
25000
50000
75000
100000
125000
150000
175000
200000
225000
250000
275000
300000
325000
350000
375000
400000
425000
450000
475000
500000
525000
550000
575000
600000
625000
650000
675000
700000
725000
750000
775000
800000
825000
850000
875000
900000
925000
950000
975000
1000000
1025000
1050000
1075000


In [80]:
print(list(G.nodes)[:10])

['296508855', '933167783931768832', '2460335661', '251657704', '1416633163363856389', '570350814', '1305169119076114432', '1212741761509908480', '1261347536218005505', '1240306312434647043']


In [100]:

G.nodes["472447110"]

{'seed': '1466829037645582341', 'username': 'JohannesNormann'}

In [84]:
G_sorted = pd.DataFrame(sorted(G.in_degree, key=lambda x: x[1], reverse=True))
G_sorted.columns = ["id","degree"]
G_sorted.head()

,id,degree
0,1132688391357837314,1923
1,1097754070570491906,1570
2,472447110,1385
3,1248887845030432768,1366
4,886701763407093760,1347


In [91]:
nx.info(G)

'DiGraph with 718662 nodes and 2405097 edges'

In [99]:
import random
k = 10000
sampled_nodes = random.sample(G.nodes, k)
sampled_graph = G.subgraph(sampled_nodes)
nx.info(sampled_graph)
nx.write_gexf(sampled_graph, "test_smallest.gexf")

In [59]:
G_tmp = nx.k_core(G, 200) 

In [60]:
nx.info(G_tmp)

'DiGraph with 3512 nodes and 808190 edges'

In [61]:
nx.write_gexf(G_tmp, "test_small.gexf")